In [58]:
import os
import os.path
import json
import ast
import pandas as pd
import numpy as np
import gzip

# **ETL**

**Extraer**

In [59]:
# Funcion para leer user_reviews y user_items
def read_multiple_json(ruta_archivo):

    # creamos una lista 'data' que almacenará cada componete del json
    data = []
    with gzip.open(ruta_archivo, 'rt', encoding='utf-8') as file:
        for line in file:
            data.append(line)
    
    # Iteramos a través de la lista 'data' y convertimos en diccionarios
    # la función eval() convierte cada cadena en un diccionario y se almacena en la lista 'resultados'
    resultados = []
    for cadena in data:
        diccionario = eval(cadena)
        resultados.append(diccionario)
    
    # convertimos la lista 'resultados' en un dataframe, tomando como columna las llaves del primer diccionario de la lista
    df = pd.DataFrame(resultados, columns= resultados[0].keys())
    return df

In [60]:
df_reviews = read_multiple_json('..\\STEAM-MLOps\\data\\user_reviews.json.gz')

In [61]:
df_items = read_multiple_json('..\\STEAM-MLOps\\data\\users_items.json.gz')

In [62]:
# Funcion para leer steam_games ya que con la primera funcion no dejaba
def read_multiple_json_nan(ruta_archivo):

    # creamos una lista 'data' que almacenará cada componete del json
    data = []
    with gzip.open(ruta_archivo, 'rt', encoding='utf-8') as file:
        for line in file:
            data.append(line)
    
    # Iteramos a través de la lista 'data' y convertimos en diccionarios
    # la función json.loads() convierte cada cadena en un diccionario y se almacena en la lista 'resultados'
    resultados = []
    for cadena in data:
        try:
            diccionario = json.loads(cadena)
        except ValueError:
            diccionario = None
        resultados.append(diccionario)
    
    # convertimos la lista 'resultados' en un dataframe, tomando como columna las llaves del primer diccionario de la lista
    df = pd.DataFrame(resultados, columns= resultados[0].keys())
    return df

In [63]:
df_games = read_multiple_json_nan('..\\STEAM-MLOps\\data\\steam_games.json.gz')

**Transformar**

steam_games

In [64]:
# Elimina solo las filas que tienen todos los valores nulos
df_games = df_games.dropna(how='all')
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


In [65]:
# Nos quedamos con columnas que necesitaremos
df_games = df_games[['id', 'title', 'genres', 'tags', 'specs', 'release_date','developer']]
df_games.head()

,id,title,genres,tags,specs,release_date,developer
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,Kotoshiro
88311,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,Secret Level SRL
88312,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,Poolians.com
88313,767400,弹炸人2222,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",[Single-player],2017-12-07,彼岸领域
88314,773570,NaN,NaN,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",NaN,NaN


In [66]:
# Cambiamos el nombre de la columna id a item_id
df_games.rename(columns={'id':'item_id'}, inplace=True)

In [67]:
# Seguimos limpiando nulos
df_games = df_games.dropna(subset=['item_id', 'title', 'release_date','tags', 'genres','specs','developer'])
df_games

,item_id,title,genres,tags,specs,release_date,developer
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,Kotoshiro
88311,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,Secret Level SRL
88312,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,Poolians.com
88313,767400,弹炸人2222,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",[Single-player],2017-12-07,彼岸领域
88315,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]","[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",2018-01-04,Trickjump Games Ltd
...,...,...,...,...,...,...,...
120439,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]","[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]",2018-01-04,Bidoniera Games
120440,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]","[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",2018-01-04,"Nikita ""Ghost_RUS"""
120441,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]","[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",2018-01-04,Sacada
120442,610660,Russian Roads,"[Indie, Racing, Simulation]","[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",2018-01-04,Laush Dmitriy Sergeevich


In [68]:
# Eliminamos item_id duplicados
df_games.drop_duplicates(subset='item_id', inplace=True)

In [69]:
# Información sobre las columnas
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28354 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   item_id       28354 non-null  object
 1   title         28354 non-null  object
 2   genres        28354 non-null  object
 3   tags          28354 non-null  object
 4   specs         28354 non-null  object
 5   release_date  28354 non-null  object
 6   developer     28354 non-null  object
dtypes: object(7)
memory usage: 1.7+ MB


In [70]:
# Columna 'item_id' lo cambiamos a tipo numérico
df_games['item_id'] = pd.to_numeric(df_games['item_id'])
# Columna 'release_date' lo cambiamos a tipo datetime y los errores a nulos
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

# Eliminamos valores nulos
df_games.dropna(inplace=True)

# Reseteamos el índice del DataFrame
df_games.reset_index(drop=True, inplace=True)
df_games

,item_id,title,genres,tags,specs,release_date,developer
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,Kotoshiro
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,Secret Level SRL
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,Poolians.com
3,767400,弹炸人2222,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",[Single-player],2017-12-07,彼岸领域
4,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]","[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",2018-01-04,Trickjump Games Ltd
...,...,...,...,...,...,...,...
28064,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]","[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]",2018-01-04,Bidoniera Games
28065,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]","[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",2018-01-04,"Nikita ""Ghost_RUS"""
28066,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]","[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",2018-01-04,Sacada
28067,610660,Russian Roads,"[Indie, Racing, Simulation]","[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",2018-01-04,Laush Dmitriy Sergeevich


In [71]:
# Nos quedamos con las siguientes columnas optimas para el desarrollo del API
games = df_games[['item_id', 'title', 'genres', 'release_date','developer']]
games

,item_id,title,genres,release_date,developer
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,Kotoshiro
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018-01-04,Secret Level SRL
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24,Poolians.com
3,767400,弹炸人2222,"[Action, Adventure, Casual]",2017-12-07,彼岸领域
4,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018-01-04,Trickjump Games Ltd
...,...,...,...,...,...
28064,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018-01-04,Bidoniera Games
28065,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018-01-04,"Nikita ""Ghost_RUS"""
28066,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018-01-04,Sacada
28067,610660,Russian Roads,"[Indie, Racing, Simulation]",2018-01-04,Laush Dmitriy Sergeevich


In [72]:
# Descomponer las listas en la columna 'genres' en filas individuales, replicando el resto de los datos asociados con esa fila.
games = games.explode('genres')

# Restablecer el índice del DataFrame. El parámetro drop=True significa que el índice anterior se eliminará.
# El parámetro inplace=True indica que los cambios se aplicarán directamente al DataFrame 'games'.
games.reset_index(drop=True, inplace=True)
games

,item_id,title,genres,release_date,developer
0,761140,Lost Summoner Kitty,Action,2018-01-04,Kotoshiro
1,761140,Lost Summoner Kitty,Casual,2018-01-04,Kotoshiro
2,761140,Lost Summoner Kitty,Indie,2018-01-04,Kotoshiro
3,761140,Lost Summoner Kitty,Simulation,2018-01-04,Kotoshiro
4,761140,Lost Summoner Kitty,Strategy,2018-01-04,Kotoshiro
...,...,...,...,...,...
69745,610660,Russian Roads,Indie,2018-01-04,Laush Dmitriy Sergeevich
69746,610660,Russian Roads,Racing,2018-01-04,Laush Dmitriy Sergeevich
69747,610660,Russian Roads,Simulation,2018-01-04,Laush Dmitriy Sergeevich
69748,658870,EXIT 2 - Directions,Casual,2017-09-02,"xropi,stev3ns"


In [73]:
# Extrayendo el año de release_date en una nueva columna year_release
games['year_release'] = games['release_date'].dt.year
games

,item_id,title,genres,release_date,developer,year_release
0,761140,Lost Summoner Kitty,Action,2018-01-04,Kotoshiro,2018
1,761140,Lost Summoner Kitty,Casual,2018-01-04,Kotoshiro,2018
2,761140,Lost Summoner Kitty,Indie,2018-01-04,Kotoshiro,2018
3,761140,Lost Summoner Kitty,Simulation,2018-01-04,Kotoshiro,2018
4,761140,Lost Summoner Kitty,Strategy,2018-01-04,Kotoshiro,2018
...,...,...,...,...,...,...
69745,610660,Russian Roads,Indie,2018-01-04,Laush Dmitriy Sergeevich,2018
69746,610660,Russian Roads,Racing,2018-01-04,Laush Dmitriy Sergeevich,2018
69747,610660,Russian Roads,Simulation,2018-01-04,Laush Dmitriy Sergeevich,2018
69748,658870,EXIT 2 - Directions,Casual,2017-09-02,"xropi,stev3ns",2017


user_reviews

In [74]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [75]:
# Eliminamos la columna "user_url"
df_reviews = df_reviews.drop('user_url', axis=1)
df_reviews

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [76]:
# Creamos una lista vacía para almacenar las filas resultantes
nuevas_filas = []

# Iteramos a través de cada fila del DataFrame 'df_reviews'
for _, row in df_reviews.iterrows():
    user_id = row['user_id']    # Guardamos el valor de la columna 'user_id'
    lista_reviews = row['reviews']  # Guardamos la lista de diccionarios de la columna 'reviews'
    
    # Iteramos a través de la lista de diccionarios 'lista_reviews'
    for diccionario in lista_reviews:
        nueva_fila = {'user_id': user_id, **diccionario}    # Creamos un diccionario combinando 'user_id' con el diccionario desempaquetado
        nuevas_filas.append(nueva_fila)     # Añadimos el diccionario 'nueva_fila' a la lista de filas resultantes

# Creamos un nuevo DataFrame 'user_reviews_desplegado' desplegando la lista de diccionarios 'nuevas_filas'
user_reviews_desplegado = pd.DataFrame(nuevas_filas)

In [77]:
user_reviews_desplegado

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59300,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59301,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59302,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [78]:
# Columna recommend a tipo int
user_reviews_desplegado['recommend'] = user_reviews_desplegado['recommend'].astype(int)

In [79]:
# Extraer el año de la columna 'posted' y almacenarlo en una nueva columna 'year_posted'
user_reviews_desplegado['year_posted'] = user_reviews_desplegado['posted'].str.extract(r'(\d{4})')
# Eliminar la columna 'posted' del DataFrame 
user_reviews_desplegado.drop('posted', axis=1, inplace=True)

user_reviews_desplegado

,user_id,funny,last_edited,item_id,helpful,recommend,review,year_posted
0,76561197970982479,,,1250,No ratings yet,1,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,,,22200,No ratings yet,1,It's unique and worth a playthrough.,2011
2,76561197970982479,,,43110,No ratings yet,1,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,,,251610,15 of 20 people (75%) found this review helpful,1,I know what you think when you see this title ...,2014
4,js41637,,,227300,0 of 1 people (0%) found this review helpful,1,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...,...,...
59300,76561198312638244,,,70,No ratings yet,1,a must have classic from steam definitely wort...,NaN
59301,76561198312638244,,,362890,No ratings yet,1,this game is a perfect remake of the original ...,NaN
59302,LydiaMorley,1 person found this review funny,,273110,1 of 2 people (50%) found this review helpful,1,had so much fun plaing this and collecting res...,NaN
59303,LydiaMorley,,,730,No ratings yet,1,:D,NaN


In [80]:
# Calcula la frecuencia relativa de cada valor único en la columna 'year_posted'
# Proporciona una visión general de la distribución de años de publicación en el DataFrame df_reviews.
user_reviews_desplegado['year_posted'].value_counts(normalize=True)

# Imputar valores nulos en 'year_posted' con la mediana (2013)
user_reviews_desplegado['year_posted'].fillna(2013, inplace=True)

# Revisar valores nulos
user_reviews_desplegado.isnull().sum()

user_id        0
funny          0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
year_posted    0
dtype: int64

In [81]:
# year_posted e item_id convertidos a dato numerico
user_reviews_desplegado['year_posted'] = pd.to_numeric(user_reviews_desplegado['year_posted'])
user_reviews_desplegado['item_id'] = pd.to_numeric(user_reviews_desplegado['item_id'])
user_reviews_desplegado

,user_id,funny,last_edited,item_id,helpful,recommend,review,year_posted
0,76561197970982479,,,1250,No ratings yet,1,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,,,22200,No ratings yet,1,It's unique and worth a playthrough.,2011
2,76561197970982479,,,43110,No ratings yet,1,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,,,251610,15 of 20 people (75%) found this review helpful,1,I know what you think when you see this title ...,2014
4,js41637,,,227300,0 of 1 people (0%) found this review helpful,1,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...,...,...
59300,76561198312638244,,,70,No ratings yet,1,a must have classic from steam definitely wort...,2013
59301,76561198312638244,,,362890,No ratings yet,1,this game is a perfect remake of the original ...,2013
59302,LydiaMorley,1 person found this review funny,,273110,1 of 2 people (50%) found this review helpful,1,had so much fun plaing this and collecting res...,2013
59303,LydiaMorley,,,730,No ratings yet,1,:D,2013


In [82]:
# Eliminar columnas innecesarias
user_reviews_desplegado.drop(columns=['funny', 'last_edited', 'helpful'], inplace=True)

In [83]:
# Damos un reset al indice
user_reviews_desplegado.reset_index(drop=True, inplace=True)
user_reviews_desplegado

,user_id,item_id,recommend,review,year_posted
0,76561197970982479,1250,1,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,22200,1,It's unique and worth a playthrough.,2011
2,76561197970982479,43110,1,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,251610,1,I know what you think when you see this title ...,2014
4,js41637,227300,1,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
59300,76561198312638244,70,1,a must have classic from steam definitely wort...,2013
59301,76561198312638244,362890,1,this game is a perfect remake of the original ...,2013
59302,LydiaMorley,273110,1,had so much fun plaing this and collecting res...,2013
59303,LydiaMorley,730,1,:D,2013


In [84]:
# Eliminar datos duplicados
user_reviews_desplegado.drop_duplicates(inplace=True)
user_reviews_desplegado

,user_id,item_id,recommend,review,year_posted
0,76561197970982479,1250,1,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,22200,1,It's unique and worth a playthrough.,2011
2,76561197970982479,43110,1,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,251610,1,I know what you think when you see this title ...,2014
4,js41637,227300,1,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
59300,76561198312638244,70,1,a must have classic from steam definitely wort...,2013
59301,76561198312638244,362890,1,this game is a perfect remake of the original ...,2013
59302,LydiaMorley,273110,1,had so much fun plaing this and collecting res...,2013
59303,LydiaMorley,730,1,:D,2013


In [85]:
# Comenzamos a construir la columna "sentiment_analysis"
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ramir\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [86]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instanciamos un objeto que analizará el sentimiento de la columna review
analizador = SentimentIntensityAnalyzer()
"""Paso 1: Creación de la columna 'polarity'

Aplicamos el método polarity_scores a la columna 'review' en nuestro DataFrame.
Método polarity_scores:

Devuelve un valor flotante que cuantifica la intensidad del sentimiento basado en el texto de entrada.
Interpretación de la polaridad:

Si polarity < 0: Sentimiento negativo.
Si polarity = 0: Sentimiento neutro.
Si polarity > 0: Sentimiento positivo."""
user_reviews_desplegado['polarity'] = user_reviews_desplegado['review'].apply(lambda r: analizador.polarity_scores(r)['compound'])

In [87]:
user_reviews_desplegado

,user_id,item_id,recommend,review,year_posted,polarity
0,76561197970982479,1250,1,Simple yet with great replayability. In my opi...,2011,0.8481
1,76561197970982479,22200,1,It's unique and worth a playthrough.,2011,0.2263
2,76561197970982479,43110,1,Great atmosphere. The gunplay can be a bit chu...,2011,0.9117
3,js41637,251610,1,I know what you think when you see this title ...,2014,0.9566
4,js41637,227300,1,For a simple (it's actually not all that simpl...,2013,0.9708
...,...,...,...,...,...,...
59300,76561198312638244,70,1,a must have classic from steam definitely wort...,2013,0.5574
59301,76561198312638244,362890,1,this game is a perfect remake of the original ...,2013,0.9786
59302,LydiaMorley,273110,1,had so much fun plaing this and collecting res...,2013,0.7827
59303,LydiaMorley,730,1,:D,2013,0.5106


In [88]:
def sentimiento(peso):
    if peso<0: # sentimiento negativo
        return 0
    elif peso>0: # sentimiento neutro
        return 2
    else: # sentimiento positivo
        return 1
    
user_reviews_desplegado.loc[:,'sentiment_analysis'] = user_reviews_desplegado['polarity'].apply(sentimiento)

In [89]:
user_reviews_desplegado

,user_id,item_id,recommend,review,year_posted,polarity,sentiment_analysis
0,76561197970982479,1250,1,Simple yet with great replayability. In my opi...,2011,0.8481,2
1,76561197970982479,22200,1,It's unique and worth a playthrough.,2011,0.2263,2
2,76561197970982479,43110,1,Great atmosphere. The gunplay can be a bit chu...,2011,0.9117,2
3,js41637,251610,1,I know what you think when you see this title ...,2014,0.9566,2
4,js41637,227300,1,For a simple (it's actually not all that simpl...,2013,0.9708,2
...,...,...,...,...,...,...,...
59300,76561198312638244,70,1,a must have classic from steam definitely wort...,2013,0.5574,2
59301,76561198312638244,362890,1,this game is a perfect remake of the original ...,2013,0.9786,2
59302,LydiaMorley,273110,1,had so much fun plaing this and collecting res...,2013,0.7827,2
59303,LydiaMorley,730,1,:D,2013,0.5106,2


In [90]:
# Eliminar las columnas review y polarity
user_reviews_desplegado.drop(columns=['review', 'polarity'], inplace=True)
user_reviews_desplegado

,user_id,item_id,recommend,year_posted,sentiment_analysis
0,76561197970982479,1250,1,2011,2
1,76561197970982479,22200,1,2011,2
2,76561197970982479,43110,1,2011,2
3,js41637,251610,1,2014,2
4,js41637,227300,1,2013,2
...,...,...,...,...,...
59300,76561198312638244,70,1,2013,2
59301,76561198312638244,362890,1,2013,2
59302,LydiaMorley,273110,1,2013,2
59303,LydiaMorley,730,1,2013,2


In [91]:
user_reviews_desplegado['user_id'] = user_reviews_desplegado['user_id'].str.strip()
user_reviews_desplegado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58431 non-null  object
 1   item_id             58431 non-null  int64 
 2   recommend           58431 non-null  int32 
 3   year_posted         58431 non-null  int64 
 4   sentiment_analysis  58431 non-null  int64 
dtypes: int32(1), int64(3), object(1)
memory usage: 2.5+ MB


user_items

In [92]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [93]:
# Nos quedamos con las columnas necesarias
df_items = df_items[['user_id', 'items']]
df_items

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...
88305,76561198323066619,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,[]
88308,76561198329548331,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [94]:
# Creamos una lista vacía para almacenar las filas resultantes
nuevas_filas = []

# Iteramos a través de las filas del DataFrame
for index, row in df_items.iterrows():
    user_id = row['user_id']
    lista_reviews = row['items']
    
    # Iteramos a través de la lista de reseñas
    for diccionario in lista_reviews:
        nueva_fila = {'user_id': user_id, **diccionario}
        nuevas_filas.append(nueva_fila)

# Creamos un nuevo DataFrame desplegado con las filas resultantes
user_items_desplegado = pd.DataFrame(nuevas_filas)

In [95]:
user_items_desplegado

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...
5153204,76561198329548331,346330,BrainBread 2,0,0
5153205,76561198329548331,373330,All Is Dust,0,0
5153206,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5153207,76561198329548331,521570,You Have 10 Seconds 2,4,4


In [96]:
# Eliminamos la columna 'playtime_2weeks'
user_items_desplegado.drop(columns='playtime_2weeks', inplace=True)

In [97]:
# Eliminamos nulos
user_items_desplegado.dropna(inplace=True)
# Eliminamos duplicados
user_items_desplegado.drop_duplicates(inplace=True)

user_items_desplegado

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...
5153204,76561198329548331,346330,BrainBread 2,0
5153205,76561198329548331,373330,All Is Dust,0
5153206,76561198329548331,388490,One Way To Die: Steam Edition,3
5153207,76561198329548331,521570,You Have 10 Seconds 2,4


In [98]:
user_items_desplegado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094092 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           object
 2   item_name         object
 3   playtime_forever  int64 
dtypes: int64(1), object(3)
memory usage: 194.3+ MB


In [99]:
# Columna item_id a tipo de datos numerico
user_items_desplegado['item_id'] = pd.to_numeric(user_items_desplegado['item_id'])
# Cambiamos el nombre item_name a title
user_items_desplegado.rename(columns={'item_name':'title'}, inplace= True)

In [100]:
# Reseteamos el indice
user_items_desplegado.reset_index(drop=True, inplace=True)

In [101]:
# Calcular el rango intercuartílico (IQR) para 'playtime_forever'
Q1 = user_items_desplegado['playtime_forever'].quantile(0.25)
Q3 = user_items_desplegado['playtime_forever'].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites para detectar outliers
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Filtrar el DataFrame para eliminar outliers
user_items_desplegado = user_items_desplegado[(user_items_desplegado['playtime_forever'] >= lower_limit) & (user_items_desplegado['playtime_forever'] <= upper_limit)]

# Reseteamos el indice
user_items_desplegado.reset_index(drop=True, inplace=True)

# Mostrar el DataFrame resultante sin outliers
user_items_desplegado

,user_id,item_id,title,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...
4330492,76561198329548331,346330,BrainBread 2,0
4330493,76561198329548331,373330,All Is Dust,0
4330494,76561198329548331,388490,One Way To Die: Steam Edition,3
4330495,76561198329548331,521570,You Have 10 Seconds 2,4


In [102]:
user_items_desplegado = user_items_desplegado.query('playtime_forever != 0')

**merge para el API**

In [103]:
# Fusionar los DataFrames 'user_items_desplegado' y 'games' usando una fusión interna
# utilizando las columnas 'item_id' y 'title' como claves de fusión
itemsxgames = pd.merge(user_items_desplegado, games, how='inner', on=['item_id', 'title'])
itemsxgames

,user_id,item_id,title,playtime_forever,genres,release_date,developer,year_release
0,76561197970982479,10,Counter-Strike,6,Action,2000-11-01,Valve,2000
1,doctr,10,Counter-Strike,93,Action,2000-11-01,Valve,2000
2,corrupted_soul,10,Counter-Strike,108,Action,2000-11-01,Valve,2000
3,WeiEDKrSat,10,Counter-Strike,328,Action,2000-11-01,Valve,2000
4,Fr0stedLine,10,Counter-Strike,580,Action,2000-11-01,Valve,2000
...,...,...,...,...,...,...,...,...
4194864,76561198107283457,354280,ChaosTower,164,Indie,2016-02-23,CashcowGames,2016
4194865,76561198107283457,354280,ChaosTower,164,Simulation,2016-02-23,CashcowGames,2016
4194866,76561198146468235,367090,Robot Roller-Derby Disco Dodgeball Soundtrack,1,Action,2015-05-01,Erik Asmussen,2015
4194867,76561198146468235,367090,Robot Roller-Derby Disco Dodgeball Soundtrack,1,Indie,2015-05-01,Erik Asmussen,2015


In [104]:
itemsxgames = itemsxgames[['genres', 'year_release', 'playtime_forever', 'user_id']]
itemsxgames

,genres,year_release,playtime_forever,user_id
0,Action,2000,6,76561197970982479
1,Action,2000,93,doctr
2,Action,2000,108,corrupted_soul
3,Action,2000,328,WeiEDKrSat
4,Action,2000,580,Fr0stedLine
...,...,...,...,...
4194864,Indie,2016,164,76561198107283457
4194865,Simulation,2016,164,76561198107283457
4194866,Action,2015,1,76561198146468235
4194867,Indie,2015,1,76561198146468235


In [105]:
# Fusionar los DataFrames 'user_reviews_desplegado' y 'games' usando una fusión interna
# utilizando la columna 'item_id' como clave de fusión
reviewsxgames = pd.merge(user_reviews_desplegado, games, how='inner', on='item_id')

reviewsxgames

,user_id,item_id,recommend,year_posted,sentiment_analysis,title,genres,release_date,developer,year_release
0,76561197970982479,1250,1,2011,2,Killing Floor,Action,2009-05-14,Tripwire Interactive,2009
1,death-hunter,1250,1,2015,2,Killing Floor,Action,2009-05-14,Tripwire Interactive,2009
2,DJKamBer,1250,1,2013,0,Killing Floor,Action,2009-05-14,Tripwire Interactive,2009
3,diego9031,1250,1,2015,1,Killing Floor,Action,2009-05-14,Tripwire Interactive,2009
4,76561198081962345,1250,1,2014,1,Killing Floor,Action,2009-05-14,Tripwire Interactive,2009
...,...,...,...,...,...,...,...,...,...,...
123387,MeloncraftLP,262850,1,2013,1,The Journey Down: Chapter Two,Indie,2014-08-25,SkyGoblin,2014
123388,vinquility,431510,1,2013,2,Mystic Destinies: Serendipity of Aeons,Adventure,2016-02-11,Aeon Dream Studios,2016
123389,vinquility,431510,1,2013,2,Mystic Destinies: Serendipity of Aeons,Casual,2016-02-11,Aeon Dream Studios,2016
123390,vinquility,431510,1,2013,2,Mystic Destinies: Serendipity of Aeons,Indie,2016-02-11,Aeon Dream Studios,2016


In [106]:
# Instalamos pyarrow para poder convertir los archivos a parquet
!pip install pyarrow

In [107]:
# Guardamos los dataframes que hicimos merge para el API
itemsxgames.to_parquet('itemsxgames.parquet', index=False)
reviewsxgames.to_parquet('reviewsxgames.parquet', index=False)

In [108]:
# Guardando los dataframe procesados

# Guardar df_games en formato parquet en la carpeta 'data'
games.to_parquet(os.path.join('data', 'steam_games.parquet'), index=False)

# Guardar user_items_desplegado en formato parquet en la carpeta 'data'
user_items_desplegado.to_parquet(os.path.join('data', 'user_items.parquet'), index=False)

# Guardar user_reviews_desplegado en formato parquet en la carpeta 'data'
user_reviews_desplegado.to_parquet(os.path.join('data', 'user_reviews.parquet'), index=False)

# **TRANFORMACIÓN PARA EL SISTEMA DE RECOMENDACION ML**

In [109]:
steamxgames = df_games
steamxgames

,item_id,title,genres,tags,specs,release_date,developer
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,Kotoshiro
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,Secret Level SRL
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,Poolians.com
3,767400,弹炸人2222,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",[Single-player],2017-12-07,彼岸领域
4,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]","[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",2018-01-04,Trickjump Games Ltd
...,...,...,...,...,...,...,...
28064,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]","[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]",2018-01-04,Bidoniera Games
28065,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]","[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",2018-01-04,"Nikita ""Ghost_RUS"""
28066,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]","[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",2018-01-04,Sacada
28067,610660,Russian Roads,"[Indie, Racing, Simulation]","[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",2018-01-04,Laush Dmitriy Sergeevich


In [110]:
# Lista de columnas a procesar
columns_to_join = ['genres', 'tags', 'specs']

# Aplicar la operación a cada elemento del DataFrame
steamxgames[columns_to_join] = steamxgames[columns_to_join].applymap(lambda lista: ', '.join(map(str, lista)))

In [111]:
#La columna 'description' se está construyendo concatenando valores de las columnas existentes
#'genres', 'tags', y 'specs' para cada fila del DataFrame
steamxgames['description'] = steamxgames.apply(lambda row: f"{row['genres']}, {row['tags']}, {row['specs']}", axis=1)

In [112]:
# Eliminamos columnas que ya no serian necesarias
steamxgames.drop(columns=['genres', 'tags', 'specs', 'release_date'], inplace=True)
steamxgames

,item_id,title,developer,description
0,761140,Lost Summoner Kitty,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy, S..."
1,643980,Ironbound,Secret Level SRL,"Free to Play, Indie, RPG, Strategy, Free to Pl..."
2,670290,Real Pool 3D - Poolians,Poolians.com,"Casual, Free to Play, Indie, Simulation, Sport..."
3,767400,弹炸人2222,彼岸领域,"Action, Adventure, Casual, Action, Adventure, ..."
4,772540,Battle Royale Trainer,Trickjump Games Ltd,"Action, Adventure, Simulation, Action, Adventu..."
...,...,...,...,...
28064,745400,Kebab it Up!,Bidoniera Games,"Action, Adventure, Casual, Indie, Action, Indi..."
28065,773640,Colony On Mars,"Nikita ""Ghost_RUS""","Casual, Indie, Simulation, Strategy, Strategy,..."
28066,733530,LOGistICAL: South Africa,Sacada,"Casual, Indie, Strategy, Strategy, Indie, Casu..."
28067,610660,Russian Roads,Laush Dmitriy Sergeevich,"Indie, Racing, Simulation, Indie, Simulation, ..."


In [113]:
# Guardamos el dataframe para el API
steamxgames.to_parquet('steamxgamesml.parquet', index=False)

**MODELO DE RECOMENDACION DE JUEGO**

In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Crear un pipeline con TfidfVectorizer y StandardScaler para normalizar los vectores TF-IDF
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=lambda x: x.split(', '))),
    ('scaler', StandardScaler(with_mean=False))
])

# Ajustar y transformar la descripción de los juegos
matriz_descripcion_tfidf = pipeline.fit_transform(steamxgames['description'])

def recomendacion_juego(id_producto: int):
    '''
    Se ingresa el id de producto (item_id) y retorna una lista con 5 juegos recomendados similares al ingresado (title).
    '''
    
    # Verifica si el ID ingresado existe en la columna 'item_id'
    if id_producto not in steamxgames['item_id'].values:
        return 'El ID no existe, intente con otro'
    else:
        # Obtiene el índice del juego con el ID ingresado
        index = steamxgames.index[steamxgames['item_id'] == id_producto][0]

        # Obtiene el vector de descripción TF-IDF del juego en cuestión
        description_tfidf_index = matriz_descripcion_tfidf[index]

        # Calcula la similitud coseno entre el vector TF-IDF del juego y los vectores TF-IDF de todos los juegos
        # Obtiene los índices de los juegos más similares, excluyendo el juego de entrada
        similarities = linear_kernel(description_tfidf_index, matriz_descripcion_tfidf).flatten()
        indices_maximos = similarities.argsort()[-2:-7:-1]

        # Crea una lista de títulos de juegos recomendados
        recomendaciones = [steamxgames['title'][i] for i in indices_maximos]

        return recomendaciones

# Ejemplo de uso
recomendacion_juego(745400)

c:\Users\ramir\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['ExZeus 2',
 'Root Letter - Artbook',
 'Wolfenstein II: The Freedom Chronicles - Episode 1',
 'Sniper Ghost Warrior 3 Original Soundtrack',
 'Sniper Ghost Warrior 3 Original Georgian Soundtrack']